In [37]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

In [38]:
def sum(data: list[float]) -> float:
    result = 0
    for item in data:
        result += item
    return result

In [39]:
def average(data: list[float]) -> float:
    return (1 / len(data)) * sum(data)

In [40]:
def standard_deviation(data: list[float]) -> float:
    deviations = []
    mean = average(data)
    for item in data:
        deviations.append(math.pow(item - mean, 2))
    deviations_sum = sum(deviations)
    return math.sqrt(deviations_sum / len(data))

In [41]:
def standard_deviation_from_the_mean(data: list[float]) -> float:
    return standard_deviation(data) / math.sqrt(len(data))

In [42]:
from ctypes import ArgumentError

def mmq_sigma_square(y_errors: list[float]) -> float:
    items_to_sum = [1 / i ** 2 for i in y_errors]
    return sum(items_to_sum)

def mmq_x(xs: list[float], y_errors: list[float]) -> float:
    if (len(xs) != len(y_errors)):
        raise ArgumentError(xs)
    items_to_sum = []
    for xi, y_error_i in zip(xs, y_errors):
        items_to_sum.append(xi / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_x_square(xs: list[float], y_errors: list[float]) -> float:
    if (len(xs) != len(y_errors)):
        raise ArgumentError(xs)
    items_to_sum = []
    for xi, y_error_i in zip(xs, y_errors):
        items_to_sum.append((xi ** 2) / (y_error_i ** 2))
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_y(ys: list[float], y_errors: list[float]) -> float:
    if (len(ys) != len(y_errors)):
        raise ArgumentError(ys, y_errors)
    items_to_sum = []
    for yi, y_error_i in zip(ys, y_errors):
        items_to_sum.append(yi / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum) 

def mmq_xy(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    if (not (len(xs) == len(ys) and len(ys) == len(y_errors))):
        raise ArgumentError(ys, xs, y_errors)
    items_to_sum = []
    for xi, yi, y_error_i in zip(xs, ys, y_errors):
        items_to_sum.append((xi * yi) / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_a(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    x = mmq_x(xs, y_errors)
    y = mmq_y(ys, y_errors)
    xy = mmq_xy(xs, ys, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return (x * y - xy) / (x ** 2 - x_square)

def mmq_b(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    return mmq_y(ys, y_errors) - mmq_a(xs, ys, y_errors) * mmq_x(xs, y_errors)

def mmq_delta_a(xs: list[float], y_errors: list[float]) -> float:
    sigma_square = mmq_sigma_square(y_errors)
    x = mmq_x(xs, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return math.sqrt((1 / sigma_square) / (x_square - x ** 2))

def mmq_delta_b(xs: list[float], y_errors: list[float]) -> float:
    sigma_square = mmq_sigma_square(y_errors)
    x = mmq_x(xs, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return math.sqrt((x_square / sigma_square) / (x_square - x ** 2))

def alternative_mmq_a(xs: list[float], ys: list[float]) -> float:
    x_average = average(xs)
    y_average = average(ys)
    y_sum = [yi - y_average for yi in ys]
    x_sum = [xi - x_average for xi in xs]
    return sum([xi * yi for xi, yi in zip(x_sum, y_sum)])/sum([xi ** 2 for xi in x_sum])

In [43]:
## Relatório 2 - FEMEC

def erro_dos_intervalos(comprimentos: list[float]) -> float:
    return math.sqrt(standard_deviation_from_the_mean(comprimentos) ** 2 + 0.05 ** 2)

def erro_cronometro(tempos: list[float]) -> float:
    return math.sqrt(standard_deviation_from_the_mean(tempos) ** 2 + 0.01 ** 2)

def erro_velocidade(l: float, erroL: float, t: float, erroT: float) -> float:
    return (l / t) * math.sqrt((erroL / l) ** 2 + (erroT / t) ** 2)
    
def row_tabela_2(intervalo: str, comprimentos: list[float], tempos: list[float]) -> None:
    print(intervalo)
    mediaComprimentos = average(comprimentos)
    incertezaComprimentos = erro_dos_intervalos(comprimentos)
    mediaTempos = average(tempos)
    incertezaTempos = erro_cronometro(tempos)
    v = mediaComprimentos / mediaTempos
    erroV = erro_velocidade(mediaComprimentos, incertezaComprimentos, mediaTempos, incertezaTempos)
    print(f"Média de comprimento = {mediaComprimentos} +- {incertezaComprimentos}")
    print(f"Média de tempo = {mediaTempos} +- {incertezaTempos}")
    print(f"Velocidade Média: {v} +- {erroV}")
    
def tabela_3_row(sensor: str, medidas_sensores: list[tuple[list[float], list[float]]]) -> None:
    media_comprimento = erro_comprimento = media_tempo = erro_tempo = tempo_square = erro_tempo_square = 0
    for medidas in medidas_sensores:
        (comprimentos, tempos) = medidas
        media_comprimento += average(comprimentos)
        erro_comprimento += erro_comprimento + erro_dos_intervalos(comprimentos) ** 2
        media_tempo += average(tempos)
        erro_tempo += erro_tempo + erro_cronometro(tempos) ** 2
        tempo_square = media_tempo ** 2
        erro_tempo_square += erro_tempo_square + (2 * erro_cronometro(tempos) / average(tempos)) ** 2
    erro_comprimento = math.sqrt(erro_comprimento)
    erro_tempo = math.sqrt(erro_tempo)
    erro_tempo_square = math.sqrt(erro_tempo_square)
    print(f"Sensor {sensor}")
    print(f"Xm = {round(media_comprimento, 3)}, Erro Xm = {round(erro_comprimento, 3)}, tm = {round(media_tempo, 3)}, Erro tm = {round(erro_tempo, 3)}, tm2 = {round(tempo_square, 3)}, Erro tm2 = {round(erro_tempo_square, 3)}")
        
l1_medidas = [15.3, 13.7, 11.5]
t1_medidas = [0.735, 0.727, 0.708]
l2_medidas = [20.0, 18.8, 16.2]
t2_medidas = [0.398, 0.398, 0.398]
l3_medidas = [19.5, 17.9, 16.21]
t3_medidas = [0.300, 0.299, 0.299]
l4_medidas = [19.4, 17.8, 16.0]
t4_medidas = [0.243, 0.244, 0.243]

row_tabela_2("L1", l1_medidas, t1_medidas)
print()
row_tabela_2("L2", l2_medidas, t2_medidas)
print()
row_tabela_2("L3", l3_medidas, t3_medidas)
print()
row_tabela_2("L4", l4_medidas, t4_medidas)
print()

tabela_3_row('1', [(l1_medidas, t1_medidas)])
print()
tabela_3_row('2', [(l1_medidas, t1_medidas), (l2_medidas, t2_medidas)])
print()
tabela_3_row('3', [(l1_medidas, t1_medidas), (l2_medidas, t2_medidas), (l3_medidas, t3_medidas)])
print()
tabela_3_row('4', [(l1_medidas, t1_medidas), (l2_medidas, t2_medidas), (l3_medidas, t3_medidas), (l4_medidas, t4_medidas)])
print()

ys = [13.5, 32.0, 50, 67.0]
xs = [0.52, 1.26, 2.02, 2.8]
y_errors = [1.0, 2.0, 2.0, 3.0]
print(f"Sigma square: {mmq_sigma_square(y_errors)}")
print(f"X: {mmq_x(xs, y_errors)}")
print(f"X square: {mmq_x_square(xs, y_errors)}")
print(f"Y: {mmq_y(ys, y_errors)}")
print(f"XY: {mmq_xy(xs, ys, y_errors)}")
print(f"a: {mmq_a(xs, ys, y_errors)}")
# print(f"a: {alternative_mmq_a(xs, ys)}")
print(f"b: {mmq_b(xs, ys, y_errors)}")
print(f"deltaA: {mmq_delta_a(xs, y_errors)}")
print(f"deltaB: {mmq_delta_b(xs, y_errors)}")

L1
Média de comprimento = 13.5 +- 0.9007712744581107
Média de tempo = 0.7233333333333333 +- 0.011947415651124758
Velocidade Média: 18.663594470046085 +- 1.2828939906276242

L2
Média de comprimento = 18.333333333333332 +- 0.9170706180652168
Média de tempo = 0.39799999999999996 +- 0.01
Velocidade Média: 46.06365159128978 +- 2.5785365080630784

L3
Média de comprimento = 17.869999999999997 +- 0.7771672206731887
Média de tempo = 0.2993333333333333 +- 0.010003703018086556
Velocidade Média: 59.69933184855234 +- 3.2743748497494405

L4
Média de comprimento = 17.733333333333334 +- 0.8034070966595718
Média de tempo = 0.24333333333333332 +- 0.010003703018086556
Velocidade Média: 72.87671232876713 +- 4.458398225480791

Sensor 1
Xm = 13.5, Erro Xm = 0.901, tm = 0.723, Erro tm = 0.012, tm2 = 0.523, Erro tm2 = 0.033

Sensor 2
Xm = 31.833, Erro Xm = 1.57, tm = 1.121, Erro tm = 0.02, tm2 = 1.257, Erro tm2 = 0.069

Sensor 3
Xm = 49.703, Erro Xm = 2.352, tm = 1.421, Erro tm = 0.03, tm2 = 2.018, Erro tm2 =